<a href="https://colab.research.google.com/github/tonyhollaar/kaggle-competitions/blob/main/kaggle_co2_emmissions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h2>Welcome to Rwanda Carbon Prediction Challenge</h2></center>
<figure>
<!-- <center><img src ="https://drive.google.com/uc?export=view&id=1hSOAfRhJ_jo-MZAjq81VYJu5bZNL7EjD" width = "800" height = '500'/> -->

*About the problem*
> The ability to accurately monitor carbon emissions is a critical step in the fight against climate change. Precise carbon readings allow researchers and governments to understand the sources and patterns of carbon mass output. While Europe and North America have extensive systems in place to monitor carbon emissions on the ground, there are few available in Africa.

*Objective of this challenge*
> The objective of this challenge is to create machine learning or a deep learning model using open-source CO2 emissions data (from Sentinel-5P satellite observations) to predict carbon emissions.

These solutions will enable governments and other actors to estimate carbon emission levels across Africa, even in places where on-the-ground monitoring is not possible.



## 1.1 Importing libraries


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 668.4 kB/s eta 0:00:00


In [3]:
# Import libraries
import pandas as pd
import numpy as np
import random
import os
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
pd.options.display.float_format = '{:.5f}'.format
pd.options.display.max_rows = None

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

#####################
# packages additional
#####################
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error, r2_score

# plotting
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats

# map
import folium
from folium.plugins import MarkerCluster

# clustering for e.g. geolocations
from sklearn.cluster import KMeans

# # my own functions
# from my_prophet_functions import preprocess_X_prophet, preprocess_data_prophet, predict_prophet, plot_actual_vs_predicted, my_metrics

## 1.2 Functions

In [4]:
def preprocess_X_prophet(X):
    """
    Preprocess the X dataframe for Prophet modeling.

    Args:
        X (pd.DataFrame): Dataset with date index and independent features.

    Returns:
        pd.DataFrame: Preprocessed X dataframe with 'ds' column and regressor columns.
    """
    if isinstance(X.index, pd.DatetimeIndex):
        X = X.reset_index()

    X_prophet = X.copy()
    X_prophet = X_prophet.rename(columns={X_prophet.columns[0]: 'ds'})
    return X_prophet

def preprocess_data_prophet(y_data):
    """
    Preprocess the given data for Prophet model
    """
    # create a deep copy of dataframe e.g. completely new copy of the DataFrame is created with its own memory space
    # This means that any changes made to the new copy will not affect the original DataFrame.
    y_data_prophet = y_data.copy(deep=True)

    # Check if the index is already a datetime index and revert it to a column if needed
    if isinstance(y_data_prophet.index, pd.DatetimeIndex):
        y_data_prophet.reset_index(inplace=True)

    # REQUIREMENT PROPHET: CREATE DATAFRAME WITH DATE COLUMN 'DS' AND 'Y' column
    y_data_prophet = pd.DataFrame({"ds": y_data_prophet.iloc[:, 0], "y": y_data_prophet.iloc[:, 1]})
    return y_data_prophet

def predict_prophet(y_train, y_test, X_train, **kwargs):
    """
    Predict future values using Prophet model.

    Args:
        y_train (pd.DataFrame): Training dataset.
        y_test (pd.DataFrame): Test dataset.
        **kwargs: Keyword arguments used to adjust the Prophet model.
            Allowed keyword arguments:
            - changepoint_prior_scale (float): Parameter for changepoint prior scale.
            - seasonality_mode (str): Parameter for seasonality mode.
            - seasonality_prior_scale (float): Parameter for seasonality prior scale.
            - holidays_prior_scale (float): Parameter for holidays prior scale.
            - yearly_seasonality (bool): Whether to include yearly seasonality.
            - weekly_seasonality (bool): Whether to include weekly seasonality.
            - daily_seasonality (bool): Whether to include daily seasonality.
            - interval_width (float): Width of the uncertainty interval.

    Returns:
        pd.DataFrame: A dataframe with the following columns:
            - Actual: The actual values of the test dataset.
            - Predicted: The predicted values of the test dataset.
            - Error: The difference between actual and predicted values.
            - Error (%): The percentage difference between actual and predicted values.
    """


    # Step 1: Preprocess Data for Prophet (required are at least 'ds' column and 'y' column)
    #######################
    # X_train_prophet = preprocess_X_prophet(X_train)
    # y_train_prophet = preprocess_data_prophet(y_train)
    # joined_train_data = pd.merge(y_train_prophet, X_train_prophet, on = 'ds')
    # #st.write('joined train data', joined_train_data) #TEST

    # X_test_prophet = preprocess_X_prophet(X_test)
    # y_test_prophet = preprocess_data_prophet(y_test)
    # joined_test_data = pd.merge(y_test_prophet, X_test_prophet, on = 'ds')
    # #st.write('joined test data', joined_test_data) # TEST

    # # merge train and test data together in 1 dataframe
    # merged_data = joined_train_data.append(joined_test_data, ignore_index = True)
    # #st.write('merged data', merged_data) # TEST



    # Step 2: Define Model
    #######################
    # get the parameters from the settings either preset or adjusted by user and user pressed submit button
    m = Prophet(changepoint_prior_scale= changepoint_prior_scale,
                seasonality_mode = seasonality_mode,
                seasonality_prior_scale = seasonality_prior_scale,
                holidays_prior_scale = holidays_prior_scale,
                yearly_seasonality = yearly_seasonality,
                weekly_seasonality = weekly_seasonality,
                daily_seasonality = daily_seasonality,
                interval_width = interval_width)

    # Step 3: Add independent features/regressors to model
    #######################
    # iterate over the names of features found from training dataset (X_train) and add them to prophet model as regressors
    for column in X_train.columns:
        m.add_regressor(column)

    # Step 4: Fit the model
    #######################
    # train the model on the data with set parameters
    #m.fit(y_train_prophet)
    m.fit(joined_train_data)

    # Step 5: Create current date range + future date range
    #######################
    # Predict on the test set
    future = m.make_future_dataframe(periods=len(y_test_prophet), freq='W')

    # step 6: Add regressors to future
    #######################
    for column in merged_data.columns:
        future[column] = merged_data[column]

    # step 7: forecast
    #######################
    forecast = m.predict(future)

    # slice the test-set of the forecast - exclude the forecast on the training set although prophet model does supply it
    # Prophet model provides it to check for overfitting the model, however to prevent user from thinking it trained on whole dataset clearer to provide forecast of test set only
    yhat_test = forecast['yhat'][-len(y_test):]
    preds_df_prophet = pd.DataFrame({'Actual': y_test_prophet['y'].values, 'Predicted': yhat_test.values}, index=y_test_prophet['ds'])

    # create column date and set the datetime index to date without the time i.e. 00:00:00
    preds_df_prophet['date'] = preds_df_prophet.index.strftime('%Y-%m-%d')

    # set the date column as index column
    preds_df_prophet = preds_df_prophet.set_index('date')

    # Calculate absolute error and add it as a new column
    preds_df_prophet['Error'] = preds_df_prophet['Predicted'] - preds_df_prophet['Actual']

    # Calculate percentage difference between actual and predicted values and add it as a new column
    preds_df_prophet['Error (%)'] = (preds_df_prophet['Error'] / preds_df_prophet['Actual'])

    return preds_df_prophet

def plot_actual_vs_predicted(df_preds, my_conf_interval):
    colors = ['#5276A7', '#ff7700']
    #my_fillcolor = 'rgba(222,235,247,0.5)' # light blue
    my_fillcolor = 'rgba(161, 217, 155, 0.5)'

    fig = px.line(df_preds, x=df_preds.index, y=['Actual', 'Predicted'], color_discrete_sequence=colors)

    fig.update_layout(
        legend_title='Legend',
        font=dict(family='Arial', size=12, color='#707070'),
        yaxis=dict(
            gridcolor='#E1E1E1',
            range=[np.minimum(df_preds['Actual'].min(), df_preds['Predicted'].min()) - (df_preds['Predicted'].max() - df_preds['Predicted'].min()),
                   np.maximum(df_preds['Actual'].max(), df_preds['Predicted'].max()) + (df_preds['Predicted'].max() - df_preds['Predicted'].min())],
            zeroline=False,
        ),
        xaxis=dict(gridcolor='#E1E1E1'),
        legend=dict(yanchor="bottom", y=0.0, xanchor="center", x=0.99,  bgcolor='rgba(0,0,0,0)')
    )

    for i, color in enumerate(colors):
        fig.data[i].line.color = color
        if fig.data[i].name == 'Predicted':
            fig.data[i].line.dash = 'dot'

    confidence = float(my_conf_interval / 100)
    alpha = 1 - confidence
    n = len(df_preds)
    t_value = stats.t.ppf(1 - alpha / 2, n - 2)
    std_error = np.sqrt(np.sum((df_preds['Actual'] - df_preds['Predicted']) ** 2) / (n - 2))
    margin_error = t_value * std_error
    upper = df_preds['Predicted'] + margin_error
    lower = df_preds['Predicted'] - margin_error

    fig.add_trace(go.Scatter(
        x=df_preds.index,
        y=upper,
        fill='tonexty',
        mode='lines',
        line_color='rgba(0,0,0,0)',
        fillcolor=my_fillcolor,
        name='Upper Confidence Interval',
        showlegend=False,
        legendgroup='confidence intervals'
    ))

    fig.add_trace(go.Scatter(
        x=df_preds.index,
        y=lower,
        fill='tonexty',
        mode='lines',
        line_color='rgba(0,0,0,0)',
        fillcolor=my_fillcolor,
        name=f'{int(my_conf_interval)}% Confidence Interval',
        legendgroup='confidence intervals'
    ))

    # Display the plot directly in the Jupyter notebook
    fig.show()

def my_metrics(my_df, model_name):
    """
    Calculates performance metrics for a given model.

    Args:
        my_df (pandas.DataFrame): DataFrame containing actual and predicted values for evaluation.
            Columns:
                - 'Actual': Actual target variable values.
                - 'Predicted': Predicted target variable values.
        model_name (str): Name of the model.

    Returns:
        mape (float): Mean Absolute Percentage Error (MAPE) for the model.
        rmse (float): Root Mean Squared Error (RMSE) for the model.
        r2 (float): R-squared (coefficient of determination) for the model.
        smape (float): Symmetric Mean Absolute Percentage Error (SMAPE) for the model.

    Note:
        - The function calculates MAPE, RMSE, R-squared, and SMAPE metrics based on the provided DataFrame.
        - MAPE is the average percentage difference between predicted and actual values.
        - RMSE is the square root of the mean squared error between predicted and actual values.
        - R-squared measures the proportion of the variance in the target variable explained by the model.
        - SMAPE is the symmetric mean absolute percentage error between predicted and actual values.
        - The function updates the 'metrics_dict' dictionary with the calculated metrics for the given model.

    Example:
        result = my_metrics(df_preds, 'MyModel')
        mape_value, rmse_value, r2_value, smape_value = result
    """
    metrics_dict = {}

    # Check if not empty dataframe
    if not my_df.empty:
        # Calculate metrics
        mape = np.mean(np.abs((my_df['Actual'] - my_df['Predicted']) / my_df['Actual']))
        smape = (1 / len(my_df)) * np.sum(np.abs(my_df['Actual'] - my_df['Predicted']) / (np.abs(my_df['Actual']) + np.abs(my_df['Predicted'])))
        mse = mean_squared_error(my_df['Actual'], my_df['Predicted'])
        rmse = np.sqrt(mse)
        r2 = r2_score(my_df['Actual'], my_df['Predicted'])

    else:
        # Set metrics equal to none if df is empty
        mape = None
        smape = None
        mse = None
        rmse = None
        r2 = None

    # Add the results to the dictionary
    metrics_dict[model_name] = {'mape': mape, 'smape': smape, 'mse': mse, 'rmse': rmse, 'r2': r2}

    return metrics_dict

## 1.3 Set seed for reproducability

In [5]:
SEED = 2023
random.seed(SEED)
np.random.seed(SEED)

## 2. Load Data

In [6]:
# Specify the subfolder name
#SUBFOLDER = 'playground-series-s3e20'
#SUBFOLDER = '/content/'
SUBFOLDER = '/content/drive/MyDrive/Colab Notebooks/kaggle_co2_emmissions/'

# Construct relative paths to the subfolder
DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), SUBFOLDER)

# Load files
train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
#test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
#samplesubmission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

## 3. Data Preprocessing

In [7]:
# First we create a unique location from lat lon
train['location'] = [str(x) + '_' + str(y) for x, y in zip(train.latitude, train.longitude)]

In [8]:
print(f"number of locations: {len(train['location'].unique())}")

number of locations: 497


### Create Map of Rwanda
based on Geodata (latitude / longitude)

In [9]:
# Create a map centered around the mean latitude and longitude
latitude_center = train['latitude'].mean()
longitude_center = train['longitude'].mean()
my_map = folium.Map(location=[latitude_center, longitude_center], zoom_start=10)

# Add markers for each unique location
marker_cluster = MarkerCluster().add_to(my_map)

for index, row in train.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    location = row['location']  # Use the unique location identifier
    folium.Marker(location=[lat, lon], popup=location).add_to(marker_cluster)

# Save the map to an HTML file and display it
my_map.save('map_with_locations.html')


## 4. Feature Engineering

### Pipeline preprocessing

In [11]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import BinaryEncoder
from sklearn.impute import SimpleImputer

# Custom Transformer to sort DataFrame
class SortDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_sorted = X.sort_values(by=['location', 'year', 'week_no'], ignore_index=True)
        return X_sorted

# Custom Transformer to convert year and week_no to datetime
class ConvertToDate(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['ds'] = pd.to_datetime(X.apply(lambda row: f"{row['year']} {row['week_no']} 0", axis=1), format="%Y %W %w")
        return X

# Custom Transformer to preprocess geolocations
class PreprocessGeolocations(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        lat_increment = 0.75
        lon_increment = 0.75
        X['lat_bin'] = (X['latitude'] // lat_increment).astype(int)
        X['lon_bin'] = (X['longitude'] // lon_increment).astype(int)
        X['location_bin_grid'] = X['lat_bin'].astype(str) + '_' + X['lon_bin'].astype(str)

        num_clusters = 60
        locations = X[['latitude', 'longitude']].values
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        X['location_bin_cluster'] = kmeans.fit_predict(locations)

        return X

# Custom Transformer to binary encode location_bin_grid column
class EncodeLocationBin(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        encoder = BinaryEncoder(cols=['location_bin_grid'], drop_invariant=True)  # Added drop_invariant
        X_encoded = encoder.fit_transform(X)  # Apply encoding to the entire DataFrame
        return X_encoded

# Custom Transformer to remove features with high missing values
class RemoveHighMissing(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8):
        self.threshold = threshold

    def fit(self, X, y=None):
        self.columns_to_drop = X.columns[X.isnull().mean() > self.threshold]
        return self

    def transform(self, X):
        X_filtered = X.drop(columns=self.columns_to_drop)
        return X_filtered

# Define the pipeline
pipeline = Pipeline([
    ('sort_dataframe', SortDataFrame()),  # Sort DataFrame as the first step
    ('convert_to_date', ConvertToDate()),
    ('preprocess_geolocations', PreprocessGeolocations()),
    ('remove_high_missing', RemoveHighMissing()),
    ('encode_location_bin', EncodeLocationBin())
])

# Fit the pipeline on the training data
pipeline.fit(train)

# Apply the fitted pipeline to the DataFrame
transformed_df = pipeline.transform(train)

# Backfill missing values in the transformed DataFrame
transformed_df.fillna(method='backfill', inplace=True)

# Only set 'ds' column as the index if it's not already the index
if 'ds' in transformed_df.columns:
    transformed_df = transformed_df.set_index('ds')

# Extract the 'emission' column and move it to index 0
if transformed_df.columns[0] != 'emission':
    emission_column = transformed_df['emission']
    transformed_df.drop('emission', axis=1, inplace=True)  # Remove the 'emission' column from the DataFrame
    transformed_df.insert(0, 'emission', emission_column)  # Insert the 'emission' column at index 0


# Sort Dataframe - already sorted?
#transformed_df.sort_values(by = ['location', 'year', 'week_no'], ignore_index = True)

In [12]:
print(transformed_df.shape)
transformed_df.head()

(79023, 78)


,emission,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,...,Cloud_solar_zenith_angle,location,lat_bin,lon_bin,location_bin_grid_0,location_bin_grid_1,location_bin_grid_2,location_bin_grid_3,location_bin_grid_4,location_bin_cluster
ds,,,,,,,,,,,,,,,,,,,,,
2019-01-06,3.75099,ID_-0.510_29.290_2019_00,-0.51000,29.29000,2019,0,-0.00011,0.60302,-0.00007,0.25567,...,30.75214,-0.51_29.29,-1,39,0,0,0,0,1,57
2019-01-13,4.02518,ID_-0.510_29.290_2019_01,-0.51000,29.29000,2019,1,0.00002,0.72821,0.00001,0.13099,...,27.25178,-0.51_29.29,-1,39,0,0,0,0,1,57
2019-01-20,4.23138,ID_-0.510_29.290_2019_02,-0.51000,29.29000,2019,2,0.00051,0.74820,0.00038,0.11002,...,26.19330,-0.51_29.29,-1,39,0,0,0,0,1,57
2019-01-27,4.30529,ID_-0.510_29.290_2019_03,-0.51000,29.29000,2019,3,-0.00008,0.67630,-0.00005,0.12116,...,28.82915,-0.51_29.29,-1,39,0,0,0,0,1,57
2019-02-03,4.34732,ID_-0.510_29.290_2019_04,-0.51000,29.29000,2019,4,-0.00008,0.67630,-0.00005,0.12116,...,22.20461,-0.51_29.29,-1,39,0,0,0,0,1,57


## PCA

In [13]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def perform_pca(X_train, num_features_pca):
    # Exclude non-numeric columns from X_train
    numeric_columns = X_train.select_dtypes(include=['number']).columns
    X_numeric = X_train[numeric_columns]

    # Normalize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_numeric)

    pca = PCA(n_components=num_features_pca)
    pca.fit(X_scaled)

    feature_names = X_numeric.columns
    sorted_idx = np.argsort(pca.explained_variance_ratio_)[::-1]
    sorted_features = feature_names[sorted_idx]
    selected_cols_pca = sorted_features.tolist()

    return sorted_features, pca, sorted_idx, selected_cols_pca

def show_pca_plot(sorted_features, pca):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=pca.explained_variance_ratio_,
        y=sorted_features,
        orientation='h',
        text=np.round(pca.explained_variance_ratio_ * 100, 2),
        textposition='auto',
        marker_color='#4715EF'
    ))
    fig.update_layout(
        xaxis_title='Explained Variance Ratio',
        yaxis_title='Feature Name'
    )
    fig.show()

# Exclude 'emission' column before running PCA
X_train_pca = transformed_df.copy(deep=True)
X_train_pca = X_train_pca.drop(columns=['emission'])

num_features_pca = 47  # The desired number of components for PCA

sorted_features, pca, sorted_idx, selected_cols_pca = perform_pca(X_train_pca, num_features_pca)
show_pca_plot(sorted_features, pca)

In [14]:
selected_cols_pca

['latitude',
 'longitude',
 'year',
 'week_no',
 'SulphurDioxide_SO2_column_number_density',
 'SulphurDioxide_SO2_column_number_density_amf',
 'SulphurDioxide_SO2_slant_column_number_density',
 'SulphurDioxide_cloud_fraction',
 'SulphurDioxide_sensor_azimuth_angle',
 'SulphurDioxide_sensor_zenith_angle',
 'SulphurDioxide_solar_azimuth_angle',
 'SulphurDioxide_solar_zenith_angle',
 'SulphurDioxide_SO2_column_number_density_15km',
 'CarbonMonoxide_CO_column_number_density',
 'CarbonMonoxide_H2O_column_number_density',
 'CarbonMonoxide_cloud_height',
 'CarbonMonoxide_sensor_altitude',
 'CarbonMonoxide_sensor_azimuth_angle',
 'CarbonMonoxide_sensor_zenith_angle',
 'CarbonMonoxide_solar_azimuth_angle',
 'CarbonMonoxide_solar_zenith_angle',
 'NitrogenDioxide_NO2_column_number_density',
 'NitrogenDioxide_tropospheric_NO2_column_number_density',
 'NitrogenDioxide_stratospheric_NO2_column_number_density',
 'NitrogenDioxide_NO2_slant_column_number_density',
 'NitrogenDioxide_tropopause_pressure'

In [15]:
def show_pca_plot(pca):
    explained_var_ratio = pca.explained_variance_ratio_
    cum_explained_var_ratio = np.cumsum(explained_var_ratio)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(1, len(explained_var_ratio)+1),
                             y=cum_explained_var_ratio,
                             mode='lines+markers',
                             name='Cumulative Explained Variance'))

    fig.add_trace(go.Scatter(x=np.arange(1, len(explained_var_ratio)+1),
                             y=explained_var_ratio,
                             mode='lines+markers',
                             name='Explained Variance'))

    fig.update_layout(
        title='Explained Variance vs. Number of Components',
        xaxis_title='Number of Components',
        yaxis_title='Explained Variance',
        showlegend=True
    )
    fig.show()

# Exclude 'emission' column before running PCA
X_train_pca = transformed_df.copy(deep=True)
X_train_pca = X_train_pca.drop(columns=['emission'])

# Your dataset (X_train) and number of features for PCA
# X_train = ...  # Your training data
num_features_pca = 20  # The desired number of components for PCA

sorted_features, pca, sorted_idx, selected_cols_pca = perform_pca(X_train_pca, num_features_pca)
show_pca_plot(pca)

In [16]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, PowerTransformer, QuantileTransformer

def perform_train_test_split(df, my_insample_forecast_steps, scaler_choice=None, numerical_features=[]):
    """
    Splits a given dataset into training and testing sets based on a user-specified test-set size.

    Args:
        df (pandas.DataFrame): A Pandas DataFrame containing the dataset to be split.
        my_insample_forecast_steps (int): An integer representing the number of rows to allocate for the test-set.
        scaler_choice (str): A string representing the type of scaler to be used. The default is None, which means no
        scaling is performed.
        numerical_features (list): A list of column names representing the numerical features to be scaled.

    Returns:
        tuple: A tuple containing the training and testing sets for both the features (X) and target variable (y),
            as well as the index of the split.

    Raises:
        ValueError: If the specified test-set size is greater than or equal to the total number of rows in the dataset.
    """
    # Check if the specified test-set size is valid
    # issue with switching data frequency e.g. testcase: daily to quarterly data commented out code for now...
# =============================================================================
#     if my_insample_forecast_steps >= len(df):
#         raise ValueError("Test-set size must be less than the total number of rows in the dataset.")
# =============================================================================
    # Initialize X_train_numeric_scaled with a default value
    X_train_numeric_scaled = pd.DataFrame() # TEST
    X_test_numeric_scaled = pd.DataFrame() # TEST
    X_numeric_scaled = pd.DataFrame() # TEST

    # Split the data into training and testing sets
    X = df.iloc[:, [0, 2] + list(range(3, df.shape[1]))]
    y = df.iloc[:, 0:2]

# =============================================================================
#     # Find the index of the 'date' column
#     date_column_index = df.columns.get_loc('date')
#     # Get the date column + all columns
#     # except the target feature which is assumed to be column after 'date' column
#     X = df.iloc[:, :date_column_index+1].join(df.iloc[:, date_column_index+2:])
#     y = df.iloc[:, date_column_index+1: date_column_index+2]
# =============================================================================

    X_train = X.iloc[:-my_insample_forecast_steps, :]
    X_test = X.iloc[-my_insample_forecast_steps:, :]
    y_train = y.iloc[:-my_insample_forecast_steps, :]
    y_test = y.iloc[-my_insample_forecast_steps:, :]

    # initialize variable
    scaler = ""

    # Scale the data if user selected a scaler choice in the normalization / standardization in streamlit sidebar
    if scaler_choice != "None":
        # check if there are numerical features in dataframe, if so run code
        if numerical_features:
            # Select only the numerical features to be scaled
            X_train_numeric = X_train[numerical_features]
            X_test_numeric = X_test[numerical_features]
            X_numeric = X[numerical_features]

            # Create the scaler based on the selected choice
            scaler_choices = {
                                "MinMaxScaler": MinMaxScaler(),
                                "RobustScaler": RobustScaler(),
                                "MaxAbsScaler": MaxAbsScaler(),
                                "PowerTransformer": PowerTransformer(),
                                "QuantileTransformer": QuantileTransformer(n_quantiles=100, output_distribution="normal")
                             }

            if scaler_choice not in scaler_choices:
                           raise ValueError("Invalid scaler choice. Please choose from: MinMaxScaler, RobustScaler, MaxAbsScaler, "
                                            "PowerTransformer, QuantileTransformer")

            scaler = scaler_choices[scaler_choice]

            # Fit the scaler on the training set and transform both the training and test sets
            X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
            X_train_numeric_scaled = pd.DataFrame(X_train_numeric_scaled, columns=X_train_numeric.columns, index=X_train_numeric.index)

            # note: you do not want to fit_transform on the test set else the distribution of the entire dataset is used and is data leakage
            X_test_numeric_scaled = scaler.transform(X_test_numeric)
            X_test_numeric_scaled = pd.DataFrame(X_test_numeric_scaled, columns=X_test_numeric.columns, index=X_test_numeric.index)

            # refit the scaler on the entire exogenous features e.g. X which is used for forecasting beyond train/test sets
            X_numeric_scaled = scaler.fit_transform(X_numeric)

            # Convert the scaled array back to a DataFrame and set the column names
            X_numeric_scaled = pd.DataFrame(X_numeric_scaled, columns=X_numeric.columns, index=X_numeric.index)

    # Replace the original
    if scaler_choice != "None":
        X_train[numerical_features] = X_train_numeric_scaled[numerical_features]
        X_test[numerical_features] = X_test_numeric_scaled[numerical_features]
        X[numerical_features] = X_numeric_scaled[numerical_features]

    # Return the training and testing sets as well as the scaler used (if any)
    return X, y, X_train, X_test, y_train, y_test, scaler

# TESTING SINGLE LOCATION - PROPHET MODEL
insample prediction

In [17]:
transformed_df.tail()

,emission,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,...,Cloud_solar_zenith_angle,location,lat_bin,lon_bin,location_bin_grid_0,location_bin_grid_1,location_bin_grid_2,location_bin_grid_3,location_bin_grid_4,location_bin_cluster
ds,,,,,,,,,,,,,,,,,,,,,
2021-12-05,29.40417,ID_-3.299_30.301_2021_48,-3.29900,30.30100,2021,48,0.00028,1.19564,0.00034,0.19131,...,32.09521,-3.299_30.301,-5,40,1,0,1,0,0,58
2021-12-12,29.18650,ID_-3.299_30.301_2021_49,-3.29900,30.30100,2021,49,0.00008,1.13087,0.00006,0.17722,...,30.77147,-3.299_30.301,-5,40,1,0,1,0,0,58
2021-12-19,29.13121,ID_-3.299_30.301_2021_50,-3.29900,30.30100,2021,50,-0.00003,0.87940,-0.00003,0.18421,...,30.71617,-3.299_30.301,-5,40,1,0,1,0,0,58
2021-12-26,28.12579,ID_-3.299_30.301_2021_51,-3.29900,30.30100,2021,51,-0.00003,0.87940,-0.00003,0.18421,...,29.11287,-3.299_30.301,-5,40,1,0,1,0,0,58
2022-01-02,27.23930,ID_-3.299_30.301_2021_52,-3.29900,30.30100,2021,52,-0.00009,0.87195,-0.00008,0.00000,...,30.24639,-3.299_30.301,-5,40,1,0,1,0,0,58


In [19]:
# TEST check columns in
len(selected_cols_pca)

20

In [30]:
# Step 1: Preprocess Data for Prophet (required are at least 'ds' column and 'y' column)

#################################
# PCA
#################################
# Exclude 'emission' column before running PCA
X_train_pca = transformed_df.copy(deep=True)
X_train_pca = X_train_pca.drop(columns=['emission'])

# retrieve the features to keep
num_features_pca = 47
sorted_features, pca, sorted_idx, selected_cols_pca = perform_pca(X_train_pca, num_features_pca)

# Insert 'emission' column at index 0 and assign the modified list to lst_pca
selected_cols_pca.insert(0, 'emission')
lst_pca = selected_cols_pca.copy()  # Create a copy of the modified list

# Create a new DataFrame with selected columns from PCA
df_prophet_all_locations = transformed_df[lst_pca].copy(deep=True)
#************************************

#################################
# 1.2 take 'ds' column as column
#################################
if 'ds' not in df_prophet_all_locations.columns:
    df_prophet_all_locations.reset_index(inplace=True)
else:
    print("'ds' column already found in the DataFrame.")

#################################
# 1.3 rename emissions to 'y'
#################################
if 'emission' in df_prophet_all_locations.columns:
  df_prophet_all_locations.rename(columns={'emission': 'y'}, inplace=True)
else:
  print("'emission' column already renamed to y")

#################################
# 1.4 remove not needed columns
#################################
if 'ID_LAT_LON_YEAR_WEEK' in df_prophet_all_locations.columns:
  column_to_drop = 'ID_LAT_LON_YEAR_WEEK'
  df_prophet_all_locations.drop(columns=[column_to_drop], inplace=True)
else:
  print("'ID_LAT_LON_YEAR_WEEK' column already dropped")

# Add Unique ID for each location / timeseries dataset
df_prophet_all_locations['location'] = [str(x) + '_' + str(y) for x, y in zip(df_prophet_all_locations.latitude, df_prophet_all_locations.longitude)]

#################################
# INITIALIZE VARIABLES
#################################
# Create an empty DataFrame to store the prediction results
all_predictions_df = pd.DataFrame()

# Create an empty DataFrame to store RMSE values
rmse_df = pd.DataFrame(columns=['Location', 'RMSE'])

#################################
# Loop/iterate over each location
#################################
for idx, location in enumerate(df_prophet_all_locations['location'].unique()):
  print(f"index:{idx}, location: {location}")

  # filter on single location the dataframe (e.g. 497 locations/dataframes total to iterate over)
  df_single_location = df_prophet_all_locations[df_prophet_all_locations['location'] == location]

  # once you have a single location you can drop it
  df_single_location = df_single_location.drop(['location'], axis = 1).fillna(0)

  # NORMALIZE
  #########################
  my_numerical_features = list(df_single_location.iloc[:, 2:].select_dtypes(include=['float', 'int']).columns)

  # exclusion list - don't normalize these!
  items_to_drop = ['latitude', 'longitude', 'year', 'week_no', 'location_bin_grid_0', 'location_bin_grid_1',
                   'location_bin_grid_2', 'location_bin_grid_3', 'location_bin_grid_4', 'location_bin_cluster', ]

  my_numerical_features = list(set(my_numerical_features) - set(items_to_drop))
  normalization_choice = 'MinMaxScaler'


  # TRAIN/TEST SPLIT
  #########################
  # 53 weeks forecast
  my_insample_forecast_steps = 53

  # Train/Test Split
  X, y, X_train, X_test, y_train, y_test, scaler = perform_train_test_split(df = df_single_location,
                                                                            my_insample_forecast_steps = my_insample_forecast_steps,
                                                                            scaler_choice = normalization_choice,
                                                                            numerical_features = my_numerical_features)
  # TRAIN DATA PER LOCATION
  X_train_prophet = preprocess_X_prophet(X_train)
  y_train_prophet = preprocess_data_prophet(y_train)
  joined_train_data = pd.merge(y_train, X_train, on = 'ds')

  # TEST DATA PER LOCATION
  X_test_prophet = preprocess_X_prophet(X_test)
  y_test_prophet = preprocess_data_prophet(y_test)
  joined_test_data = pd.merge(y_test_prophet, X_test_prophet, on = 'ds')

  merged_data = joined_train_data.append(joined_test_data, ignore_index = True)

  # [OPTIONAL] Hyper-parameter tuning?

  # Step 2: Define Model
  #########################
  my_conf_interval = 80
  my_interval_width = int(my_conf_interval/100)
  changepoint_prior_scale = 1 # changed from 0.01 to 0.1
  seasonality_mode = "additive" #additive #multiplicative
  seasonality_prior_scale = 0.01
  holidays_prior_scale = 0.01
  yearly_seasonality = True
  weekly_seasonality = True
  daily_seasonality = True
  interval_width = my_interval_width

  # get the parameters from the settings either preset or adjusted by user and user pressed submit button
  m = Prophet(changepoint_prior_scale= changepoint_prior_scale,
              seasonality_mode = seasonality_mode,
              seasonality_prior_scale = seasonality_prior_scale,
              holidays_prior_scale = holidays_prior_scale,
              yearly_seasonality = yearly_seasonality,
              weekly_seasonality = weekly_seasonality,
              daily_seasonality = daily_seasonality,
              interval_width = interval_width)

  # Step 3: Add independent features/regressors to model
  #########################
  # iterate over the names of features found from training dataset (X_train) and add them to prophet model as regressors
  for column in X_train.columns:
    if column != 'ds':
      m.add_regressor(column)
    #m.add_country_holidays(country_name = get_state("ENGINEER_PAGE_COUNTRY_HOLIDAY", "country_code"))

  # Step 4: Fit the model
  #######################
  # train the model on the data with set parameters
  # fit on e.g. joined_train_data
  m.fit(joined_train_data)

  # Step 5: Create current date range + future date range
  #######################
  # Predict on the test set
  #future = m.make_future_dataframe(periods=len(y_test_prophet), freq='D')
  future = m.make_future_dataframe(periods=len(y_test_prophet), freq='W')

  # step 6: Add regressors to future
  #######################
  for column in merged_data.columns:
     future[column] = merged_data[column]
  # for column in df_single_location.columns:
  #     future[column] = df_single_location[column]

  # step 7: forecast
  #######################
  forecast = m.predict(future)

  #stop at first location to test code
  #print('forecast', forecast.head())

  yhat_test = forecast['yhat'][-len(y_test):]
  preds_df_prophet = pd.DataFrame({'Actual': y_test_prophet['y'].values, 'Predicted': yhat_test.values}, index=y_test_prophet['ds'])

  # Convert the 'ds' index to datetime if it's not already
  preds_df_prophet.index = pd.to_datetime(preds_df_prophet.index)

  # Now you can use strftime on the datetime index
  preds_df_prophet['date'] = preds_df_prophet.index.strftime('%Y-%m-%d')

  # set the date column as index column
  preds_df_prophet = preds_df_prophet.set_index('date')

  # Calculate absolute error and add it as a new column
  preds_df_prophet['Error'] = preds_df_prophet['Predicted'] - preds_df_prophet['Actual']
  # Calculate percentage difference between actual and predicted values and add it as a new column
  preds_df_prophet['Error (%)'] = (preds_df_prophet['Error'] / preds_df_prophet['Actual'])

  ##############
  # POST-PROCESS
  ##############
  # add the date and ID
  start_index = len(df_single_location) - my_insample_forecast_steps
  last_53_rows = df_single_location[['latitude', 'longitude', 'year', 'week_no']].iloc[start_index:, :]

  #last_53_rows['ds'] = pd.to_datetime(last_53_rows.apply(lambda row: f"{row['year']} {row['week_no']} 0", axis=1), format="%Y %u %w")
  last_53_rows['date'] = pd.to_datetime(last_53_rows.apply(lambda row: f"{int(row['year'])} {int(row['week_no'])} 0", axis=1), format="%Y %W %w")

  # Set the 'date' column as the index
  last_53_rows.set_index('date', inplace=True)
  preds_df_prophet[['latitude', 'longitude', 'year', 'week_no']] = last_53_rows[['latitude', 'longitude', 'year', 'week_no']]
  # recreate column for id again
  preds_df_prophet['ID_LAT_LON_YEAR_WEEK'] = (
                                                  'ID_' +
                                                  preds_df_prophet['latitude'].round(3).astype(str) +
                                                  '_' +
                                                  preds_df_prophet['longitude'].round(3).astype(str) +
                                                  '_' +
                                                  preds_df_prophet['year'].astype(int).astype(str) +
                                                  '_' +
                                                  preds_df_prophet['week_no'].apply(lambda x: f"{x:02d}")
                                              )

  preds_df_prophet['location'] = [str(x) + '_' + str(y) for x, y in zip(preds_df_prophet['latitude'], preds_df_prophet['longitude'])]

  ##########################################################################################################################################
  # now i want to add below on each loop/iteration the preds_df_prophet added to a dataframe so i get the 497 prediction dataframes appended
  # and i want printed out the RMSE score for each of the 497 prediction dataframes with the variable e.g. idx and location
  ##########################################################################################################################################
  # Append the predictions for this location to the overall DataFrame
  all_predictions_df = all_predictions_df.append(preds_df_prophet, ignore_index=True)

  # Calculate and print RMSE for this location's predictions
  rmse = np.sqrt(np.mean(preds_df_prophet['Error']**2))

  # Append RMSE value and location to the rmse_df DataFrame
  rmse_df = rmse_df.append({'Location': location, 'RMSE': rmse}, ignore_index=True)

  print(f"RMSE for location {idx} {location}: {rmse}")

rmse_df_sorted = rmse_df.sort_values(by='RMSE', ascending=False)

'ID_LAT_LON_YEAR_WEEK' column already dropped
index:0, location: -0.51_29.29


DEBUG:cmdstanpy:input tempfile: /tmp/tmpdz3435ak/o6ut5o3o.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdz3435ak/dnjumq7k.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=67594', 'data', 'file=/tmp/tmpdz3435ak/o6ut5o3o.json', 'init=/tmp/tmpdz3435ak/dnjumq7k.json', 'output', 'file=/tmp/tmpdz3435ak/prophet_modelwsql7cp6/prophet_model-20230810170627.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:06:27 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:06:28 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


RMSE for location 0 -0.51_29.29: 0.6071876485397056
index:1, location: -0.528_29.472


DEBUG:cmdstanpy:input tempfile: /tmp/tmpdz3435ak/i0_ezy6s.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdz3435ak/rwe3m89r.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=96151', 'data', 'file=/tmp/tmpdz3435ak/i0_ezy6s.json', 'init=/tmp/tmpdz3435ak/rwe3m89r.json', 'output', 'file=/tmp/tmpdz3435ak/prophet_modeltvkp5rl5/prophet_model-20230810170629.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:06:29 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:06:29 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


RMSE for location 1 -0.528_29.472: 0.27894488217511887
index:2, location: -0.547_29.653


...

RMSE for location 494 -3.174_29.926: 3.948250084767974
index:495, location: -3.287_29.713


DEBUG:cmdstanpy:input tempfile: /tmp/tmpdz3435ak/15dmfl2_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=83462', 'data', 'file=/tmp/tmpdz3435ak/o1k9nknm.json', 'init=/tmp/tmpdz3435ak/15dmfl2_.json', 'output', 'file=/tmp/tmpdz3435ak/prophet_modeljmv7n7aa/prophet_model-20230810171533.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:15:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:15:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdz3435ak/uz7skxts.json


RMSE for location 495 -3.287_29.713: 9.905807281728984
index:496, location: -3.299_30.301


DEBUG:cmdstanpy:input tempfile: /tmp/tmpdz3435ak/7bjfqdn5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=65549', 'data', 'file=/tmp/tmpdz3435ak/uz7skxts.json', 'init=/tmp/tmpdz3435ak/7bjfqdn5.json', 'output', 'file=/tmp/tmpdz3435ak/prophet_modeld476ft_t/prophet_model-20230810171534.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:15:34 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:15:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


RMSE for location 496 -3.299_30.301: 2.300529048600426


In [31]:
# Error Score
# changepoint_prior_scale = 0.01 -> RMSE: 64.55689734845194 #multiplicative
# changepoint_prior_scale = 0.1 -> RMSE: 52.6953900259546 #multiplicative
# changepoint_prior_scale = 1 -> 52.71536726261413 #multiplicative
rmse = np.sqrt(np.mean(all_predictions_df['Error']**2))
print(rmse)

45.143151414971264


In [24]:
rmse_df_sorted

,Location,RMSE
292,-2.079_29.321,1233.82063
356,-2.378_29.222,291.02009
391,-2.599_29.801,119.21828
168,-1.514_29.686,107.84874
42,-0.956_30.744,105.78875
355,-2.376_29.924,103.72728
161,-1.486_29.614,97.61009
338,-2.293_29.507,90.98634
135,-1.342_31.358,89.88374
368,-2.441_28.959,88.96075


In [ ]:
#all_predictions_df['location'] = [str(x) + '_' + str(y) for x, y in zip(all_predictions_df.latitude, all_predictions_df.longitude)]

In [ ]:
# combined dataframe which has all predictions (insample)
all_predictions_df.head()

all_predictions_df['Error_ABS'] = all_predictions_df['Error'].abs()

In [ ]:
import plotly.express as px

fig = px.line(all_predictions_df, x='week_no', y='Actual', color='location', title='Actual Values for Different Locations')
fig.update_xaxes(title='Week Number')
fig.update_yaxes(title='Actual')
fig.update_layout(legend_title='Location')
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(all_predictions_df, x='week_no', y='Error', color='location', title='Error for Different Locations')
fig.update_xaxes(title='Week Number')
fig.update_yaxes(title='Error')
fig.update_layout(legend_title='Location')
fig.show()

In [ ]:
all_predictions_df.head(5)

,Actual,Predicted,Error,Error (%),latitude,longitude,year,week_no,ID_LAT_LON_YEAR_WEEK,location,Error_ABS
0,3.31956,4.01167,0.69211,0.20850,-0.51000,29.29000,2021,0,ID_-0.51_29.29_2021_00,-0.51_29.29,0.69211
1,3.97181,4.04419,0.07238,0.01822,-0.51000,29.29000,2021,1,ID_-0.51_29.29_2021_01,-0.51_29.29,0.07238
2,4.03077,3.56913,-0.46164,-0.11453,-0.51000,29.29000,2021,2,ID_-0.51_29.29_2021_02,-0.51_29.29,0.46164
3,4.08317,4.01045,-0.07273,-0.01781,-0.51000,29.29000,2021,3,ID_-0.51_29.29_2021_03,-0.51_29.29,0.07273
4,4.16175,3.41746,-0.74429,-0.17884,-0.51000,29.29000,2021,4,ID_-0.51_29.29_2021_04,-0.51_29.29,0.74429


In [ ]:
# absolute error plot
fig = px.line(all_predictions_df, x='week_no', y='Error_ABS', color='location', title='Error (Absolute) for Different Locations')
fig.update_xaxes(title='Week Number')
fig.update_yaxes(title='Error (Absolute)')
fig.update_layout(legend_title='Location')
fig.show()

In [ ]:
df_prophet_all_locations.head()
# emissions total dataset

fig = px.line(df_prophet_all_locations, x='week_no', y='y', color='location', title='Emissions for Different Locations')
fig.update_xaxes(title='Week Number')
fig.update_yaxes(title='emission')
fig.update_layout(legend_title='Location')
fig.show()

# FORECAST 2022 -> 48 weeks into future based on test set

In [ ]:
# step 1: load test-set / define test-set
############################################
test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

# step 2: preprocess test-set in same fashion as train set
#################################################################
# Fit the pipeline on the training data
pipeline.fit(test)

# Apply the fitted pipeline to the DataFrame
transformed_df_test = pipeline.transform(test)

# Backfill missing values in the transformed DataFrame
transformed_df_test.fillna(method='backfill', inplace=True)

# Only set 'ds' column as the index if it's not already the index
if 'ds' in transformed_df_test.columns:
    transformed_df_test = transformed_df_test.set_index('ds')

# Extract the 'emission' column and move it to index 0
if transformed_df_test.columns[0] != 'emission':
    emission_column = transformed_df_test['emission']
    transformed_df_test.drop('emission', axis=1, inplace=True)  # Remove the 'emission' column from the DataFrame
    transformed_df_test.insert(0, 'emission', emission_column)  # Insert the 'emission' column at index 0

# step 3: do not run PCA but keep features found from PCA and filter features
############################################################################
# Create a new DataFrame with selected columns from PCA
df_prophet_all_locations_test = transformed_df_test[lst_pca].copy(deep=True)






# step 4: define trained Prophet model
############################################################################
#m.

# step 5: forecast 48 weeks per location / loop through each location
############################################################################
for idx, location in enumerate(df_prophet_all_locations_test['location'].unique()):
  print(f"index:{idx}, location: {location}")

  # filter on single location the dataframe (e.g. 497 locations/dataframes total to iterate over)
  df_single_location_test = df_prophet_all_locations_test[df_prophet_all_locations_test['location'] == location]

  # once you have a single location you can drop it
  df_single_location_test = df_single_location_test.drop(['location'], axis = 1).fillna(0)

  # NORMALIZE NUMERICAL FEATURES
  #########################
  my_numerical_features = list(df_single_location_test.iloc[:, 2:].select_dtypes(include=['float', 'int']).columns)

  # exclusion list - don't normalize these!
  items_to_drop = ['latitude', 'longitude', 'year', 'week_no', 'location_bin_grid_0', 'location_bin_grid_1',
                   'location_bin_grid_2', 'location_bin_grid_3', 'location_bin_grid_4', 'location_bin_cluster', ]

  my_numerical_features_test = list(set(my_numerical_features_test) - set(items_to_drop))
  normalization_choice = 'MinMaxScaler'

  ### NORMALIZE (e.g. min max data)

  # TRAIN/TEST SPLIT
  #########################
  # 53 weeks forecast
  my_insample_forecast_steps = 48

  #######################
  # Predict on the test set
  future_test = m.make_future_dataframe(periods=len(df_single_location_test), freq='W')

  # add regressors
  #######################
  for column in merged_data.columns:
     future_test[column] = merged_data[column]

  # forecast
  #######################
  forecast_test = m.predict(future_test)

  break

# step 6: post-process
############################################################################

# step 7: calculate RMSE
############################################################################

# step 8: only save the ID and prediction to dataframe
############################################################################

# step 9: save dataframe
############################################################################

# step 10: upload to kaggle / submission
############################################################################
# https://www.kaggle.com/competitions/playground-series-s3e20/overview/evaluation




In [ ]:
forecast_test